# 目的
（ほぼ）１秒ごとに取得されたtickerのデータセットに内在する何かしらの相関を探る。   
このノートでは、ある時刻までのbest bid & best askの値と相関があるのかどうかチェックする。

In [ ]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from scipy.signal import lfilter

%matplotlib inline

In [ ]:
def calc_EMA(x, N):
    """calc_EMA(x, N) -> array-like
    calculate exponential moving average (EMA)
    """
    return _calc_EMA(x, 2./(N + 1.))

def _calc_EMA(x, alpha):
    """ _calc_EMA(x, alpha) -> array-like
    Adopted from https://qiita.com/toyolab/items/6872b32d9fa1763345d8
    """
    y,_ = lfilter([alpha], [1,alpha-1], x, zi=[x[0]*(1-alpha)])
    return y

In [ ]:
def ccf(y, y2=None, mode='full'):
    if y2 is None:
        y2 = y
    y_acf = np.correlate(y, y2, mode=mode)
    if mode == 'full':
        y_acf = y_acf[y_acf.size//2:]
    y_acf /= len(y_acf)
    y_sqr = np.mean(y**2)
    return y_acf/y_sqr - 1.

In [ ]:
fldrpath = r"C:\Users\take9\Desktop\data_"
filelist = glob.glob(os.path.join(fldrpath, "*.csv"))
print(len(filelist))

In [ ]:
data = pd.read_csv(filelist[1], index_col=0)
print(data.columns)

# Best ask and bid

In [ ]:
plt.figure(figsize=(18,36))
for ii in range(36):
    plt.subplot(12, 3, ii + 1)
    data_ = data["ltp"].values[ii*100:(ii+1)*100]
#     best_ask_ = data["best_ask"].values[ii*100:(ii+1)*100] * data["best_ask_size"].values[ii*100:(ii+1)*100]
#     best_bid_ = data["best_bid"].values[ii*100:(ii+1)*100] * data["best_bid_size"].values[ii*100:(ii+1)*100]
    best_ask_ = data["best_ask"].values[ii*100:(ii+1)*100]
    best_bid_ = data["best_bid"].values[ii*100:(ii+1)*100]
    plt.plot(data_ - data_[0], "#E74C3C")
    ax2 = plt.gca().twinx()
    ax2.plot(np.arange(0, 100), best_ask_ - best_bid_, "#16A085")
    ax2.hlines(0, 100, 0, linestyles="--")
    plt.grid()
    break
plt.tight_layout()

相関をとってみる。

In [ ]:
plt.figure(figsize=(18,36))
for ii in range(36):
    plt.subplot(12, 3, ii + 1)
    data_ = data["ltp"].values[ii*100:(ii+1)*100]
#     best_ask_ = data["best_ask"].values[ii*100:(ii+1)*100] * data["best_ask_size"].values[ii*100:(ii+1)*100]
#     best_bid_ = data["best_bid"].values[ii*100:(ii+1)*100] * data["best_bid_size"].values[ii*100:(ii+1)*100]
    best_ask_ = data["best_ask"].values[ii*100:(ii+1)*100]
    best_bid_ = data["best_bid"].values[ii*100:(ii+1)*100]
    plt.plot(data_ - data_[0], "#E74C3C")
    ax2 = plt.gca().twinx()
    ax2.plot(np.arange(0, 100), ccf(data_, best_ask_ - best_bid_), "#16A085")
#     ax2.hlines(0, 100, 0, linestyles="--")
    plt.grid()
    break
plt.tight_layout()

In [ ]:
points = 10

plt.figure(figsize=(18,36))
for ii in range(36):
    plt.subplot(12, 3, ii + 1)
    data_ = data["ltp"].values[ii*100+points:(ii+1)*100+points]
#     best_ask_ = data["best_ask"].values[ii*100:(ii+1)*100] * data["best_ask_size"].values[ii*100:(ii+1)*100]
#     best_bid_ = data["best_bid"].values[ii*100:(ii+1)*100] * data["best_bid_size"].values[ii*100:(ii+1)*100]
    best_ask_ = data["best_ask"].values[ii*100:(ii+1)*100]
    best_bid_ = data["best_bid"].values[ii*100:(ii+1)*100]
    plt.plot(data_ - data_[0], "#E74C3C")
    ax2 = plt.gca().twinx()
    ax2.plot(np.arange(0, 100), ccf(data_, best_ask_ - best_bid_), "#16A085")
#     ax2.hlines(0, 100, 0, linestyles="--")
    plt.grid()
    break
plt.tight_layout()

相関なさそう。

In [ ]:
3.52 * 6.02e23 / 131.293